In [1]:
import pandas as pd
print(pd.__version__)
df1 = pd.read_excel('학교별 교육편제단위 정보_20251002기준.xlsx', skiprows = 4 )
df2 = pd.read_excel('교육과정_대학(20251020).xlsx', skiprows = 8)

2.2.3


In [2]:
#교육과정과 학과학부 데이터 결합
df2 = df2.rename(columns={
    "차수" : "조사차수",
    "본분교명": "대학구분",
    "학교구분" : "본분교",
    "학부·과(전공)코드": "학교별학과코드",
    "주야구분명" : "주야간구분",
    "학부특성명" : "학과특성",

})

In [3]:
df2.columns

Index(['조사년도', '조사차수', '학교코드', '학교명', '본분교', '대학구분', '단과대학코드', '단과대학명',
       '학교별학과코드', '학부·과(전공)명', '주야구분코드', '주야간구분', '학부특성코드', '학과특성', '학과상태',
       '교육과정', '이수구분', '학점', '교과목해설'],
      dtype='object')

In [4]:
# 1️⃣ 키 컬럼 전처리 (공백/괄호 제거, 타입 통일)
for df in [df1, df2]:
    df['학교코드'] = df['학교코드'].astype(str).str.strip()
    df['학교별학과코드'] = df['학교별학과코드'].astype(str).str.strip()
    df['학부·과(전공)명'] = (
        df['학부·과(전공)명']
        .astype(str)
        .str.replace(r'\s+', '', regex=True)       # 모든 공백 제거
        .str.replace(r'\(.*?\)', '', regex=True)   # 괄호 및 내용 제거 (선택)
    )

# 2️⃣ df2에서 필요한 열만 선택
attach_cols = ['교육과정','이수구분','학점','교과목해설']
df2_slim = df2[['학교코드','학교별학과코드','학부·과(전공)명'] + attach_cols].copy()

# 3️⃣ 조인
merged = pd.merge(
    df1,
    df2_slim,
    on=['학교코드','학교별학과코드','학부·과(전공)명'],
    how='left',      # 학과정보(df1)를 기준으로
    validate='1:m'   # 한 학과에 여러 과목이 붙는 구조
)

print(len(merged))
merged


476878


,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,소계열분류,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설
0,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
1,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,N.C.E.,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
2,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
3,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
4,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476873,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476874,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,석사,서울,서초구,NaN,NaN,NaN,NaN
476875,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476876,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,N.C.E.,인문・사회계열,2년,석사,서울,서초구,NaN,NaN,NaN,NaN


In [5]:
merged.to_excel('C:\\Users\\heeyo\\Desktop\\KERIS_2종\\교육편제단위_표준분류체계 결합.xlsx', index=False)

In [6]:
merged['대계열분류'].unique()

array(['공학계열', '자연과학계열', '인문사회계열', '예체능계열', '광역계열', '의학계열'], dtype=object)

## 교육과정 정제
- 뒤에 숫자로 나뉘어져 있는 과목 통일(초급중국어1, 초급중국어2 -> 초급중국어)
- 한영 표기 동시에 되어 있는 과목 숫자 제거 후 한글 표현으로 통일 (ex. 한국어글쓰기와 표현1 (Korean writing and expression 1))

In [ ]:
import re
import pandas as pd

# 1️⃣ 함수 정의
def clean_course_name(name):
    if pd.isna(name):
        return name
    
    # 문자열로 변환 후 양쪽 공백 제거
    name = str(name).strip()
    name = re.sub(r'\[.*?\]', '', name)
    
    # (1) 문자열 끝의 숫자, -숫자, .숫자 제거  ex) "언어심리-Ⅰ" → "언어심리"
    name = re.sub(r'\d+([\-\.]\d+)?$', '', name)
    
    # (2) 특정 특수문자 제거 (하이픈, #, @, ☆, 전각문자, ' 등)
    name = re.sub(r"['\"\-@#☆－ⅠⅡⅢⅣⅤⅥⅦⅧⅨⅩ*.‘’ｏ]", '', name) #ＳＭＡＴＩＵＮＩＸ
    
    # (3) 남은 공백 제거 (문자열 끝이나 중복 공백)
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name

before = merged['교육과정'].copy()
# 2️⃣ '교육과정' 열에 적용
merged['교육과정'] = merged['교육과정'].apply(clean_course_name)

changed_rows = (before != merged['교육과정']).sum()

# 5️⃣ 결과 출력
print(f"정제 과정에서 값이 변경된 행 수: {changed_rows:,}개")

# 3️⃣ 결과 확인
merged


정제 과정에서 값이 변경된 행 수: 155,365개


,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,소계열분류,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설
0,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
1,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,N.C.E.,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
2,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
3,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
4,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,정보・통신공학,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476873,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476874,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,석사,서울,서초구,NaN,NaN,NaN,NaN
476875,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,종교학,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN
476876,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,N.C.E.,인문・사회계열,2년,석사,서울,서초구,NaN,NaN,NaN,NaN


In [10]:
# 우선 전체 과목 목록 확보
all_courses = merged['교육과정'].dropna().unique().tolist()

# 1️⃣ 영어명 → 한글명 매핑 사전 생성
#    (괄호가 포함된 항목에서 추출)
eng2kor = {}
for c in all_courses:
    if '(' in c and ')' in c:
        kor = re.sub(r'\(.*?\)', '', c).strip()
        inside = re.findall(r'\((.*?)\)', c)
        if inside:
            inside_text = inside[0].strip()
            # 영어가 포함되어 있으면 사전 등록
            if re.search(r'[A-Za-z]', inside_text):
                # 괄호 속 영어명만 남기기 (숫자 제거)
                eng = re.sub(r'\d+([\-\.]\d+)?$', '', inside_text).strip()
                eng2kor[eng.lower()] = kor

print("생성된 영어→한글 매핑 개수:", len(eng2kor))

# 2️⃣ 정제 함수 정의
def clean_course_name_v3(name):
    if pd.isna(name):
        return name
    name = str(name).strip()
    name = name.replace('（','(').replace('）',')')

    # (1) 중첩 괄호 모두 제거
    while re.search(r'\([^()]*\)', name):
        name = re.sub(r'\([^()]*\)', '', name)
    name = name.strip()

    # (2) 끝의 숫자/-.숫자 제거
    name = re.sub(r'\d+([\-\.]\d+)?$', '', name).strip()

    # (3) 영어→한글 자동 치환 (대소문자 무시)
    lower_name = name.lower()
    for eng, kor in eng2kor.items():
        if eng in lower_name:
            name = kor
            break

    # (4) 공백 정리
    name = re.sub(r'\s+', ' ', name).strip()
    return name

# 3️⃣ 적용
before = merged['교육과정'].copy()
merged['교육과정'] = merged['교육과정'].apply(clean_course_name_v3)

# 4️⃣ 변경 행 비교
changed = merged[before != merged['교육과정']][['학교명','학부·과(전공)명','교육과정']].copy()
changed['변경_전'] = before[before != merged['교육과정']]
print("변경된 행 수:", len(changed))
display(changed.head(10))


생성된 영어→한글 매핑 개수: 11414
변경된 행 수: 124644


,학교명,학부·과(전공)명,교육과정,변경_전
0,ICT폴리텍대학,AI네트워크학과,NaN,NaN
1,ICT폴리텍대학,AI소프트웨어학과,NaN,NaN
2,ICT폴리텍대학,IT융합학과,NaN,NaN
3,ICT폴리텍대학,정보보안학과,NaN,NaN
4,ICT폴리텍대학,정보통신학과,NaN,NaN
5,가야대학교 보건대학원(김해),간호학전공,NaN,NaN
6,가야대학교 보건대학원(김해),방사선학,NaN,NaN
7,가야대학교 보건대학원(김해),언어병리학전공,NaN,NaN
8,가야대학교 보건대학원(김해),작업치료학전공,NaN,NaN
9,가야대학교 일반대학원(김해),보석디자인공학전공,NaN,NaN


In [11]:
merged['표본수 n'] = (
    merged.groupby('교육과정')['교육과정'].transform('count').astype('Int64')
)

merged

,조사년도,조사차수,학교명,학교코드,대학구분,본분교,학교구분,(대학)지역,단과대학명,단과대학코드,...,대학자체대계열,수업연한,학위과정,(학과)소재지,(학과)소재지\n(상세),교육과정,이수구분,학점,교과목해설,표본수 n
0,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
1,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
2,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
3,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
4,2025,2,ICT폴리텍대학,579,전문대학,본교,기능대학,경기,단과대구분없음,999,...,공학계열,2년,전문학사,경기,광주시,NaN,NaN,NaN,NaN,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
476873,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN,<NA>
476874,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,석사,서울,서초구,NaN,NaN,NaN,NaN,<NA>
476875,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,3년,"석사, 박사",서울,서초구,NaN,NaN,NaN,NaN,<NA>
476876,2025,2,횃불트리니티신학대학원대학교,648,대학원대학,본교,전문대학원,서울,단과대구분없음,999,...,인문・사회계열,2년,석사,서울,서초구,NaN,NaN,NaN,NaN,<NA>


In [16]:
merged.to_excel('과정 전체 데이터_20251021.xlsx')

In [17]:
nce_match = merged[(merged['대학구분'] == '대학') & (merged['학교구분'] == '대학교') & (merged['본분교'] == '본교') & (merged['(대학)지역'] == '서울')
          & (merged['주야간구분'] == '주간') & (merged['학과특성'] == '일반과정') & (merged['학과상태'] != '폐지') & (merged['수업연한'] == '4년')].copy() # 데이터 2070개 남음

In [18]:
nce_match.to_excel('nce매칭 필요 데이터_20251021.xlsx')

In [19]:
nce_match['대계열분류'].unique()

array(['인문사회계열', '자연과학계열', '예체능계열', '공학계열', '광역계열', '의학계열'], dtype=object)

In [20]:
unique_pairs = nce_match[['학교명', '학부·과(전공)명']].drop_duplicates()
unique_pairs

,학교명,학부·과(전공)명
8475,감리교신학대학교,신학부
10409,강서대학교,AI기반경영학과
10443,강서대학교,G2빅데이터경영학과
10500,강서대학교,간호학과
10567,강서대학교,글로벌경영학과
...,...,...
475146,홍익대학교,교육학과
475190,홍익대학교,국어교육과
475241,홍익대학교,수학교육과
475272,홍익대학교,역사교육과


In [21]:
unique_pairs = merged[['학교명', '학부·과(전공)명']].drop_duplicates()
unique_pairs

,학교명,학부·과(전공)명
0,ICT폴리텍대학,AI네트워크학과
1,ICT폴리텍대학,AI소프트웨어학과
2,ICT폴리텍대학,IT융합학과
3,ICT폴리텍대학,정보보안학과
4,ICT폴리텍대학,정보통신학과
...,...,...
476873,횃불트리니티신학대학원대학교,기독교학과
476874,횃불트리니티신학대학원대학교,목회학과
476875,횃불트리니티신학대학원대학교,신학과
476876,횃불트리니티신학대학원대학교,예배음악학과
